# KPIer

- Vaccination över tid och påverkan på dödsfall och intensivvård: Minskade dödsfall och svårt sjuka med vaccination?
- Jämförelse av fall i olika kommuner: Var det värst i de största städerna?
- Intensivvård mot dödsfall: Hur effektiv var vården?
- Åldersskillnaden i fall: Var valet att vaccinera äldre först rätt?



## Explorativ analys

Började med en manuell lätt avläsning av de olika excel bladen för att få en idé av vilken data som fanns och vilka KPIer som skulle kunna göra från den.

Bladed vaccinationer tidsserie ger en bild av antalet vaccinationer över tid vilket skulle kunna jämnföras med olika coviddata som dödsfall för att få en idé om hur effektivt vaccinet var. 

Det fanns också olika data från kommuner både kring vaccinationer och fall, möjliga undersökningar som skulle kunna göras är om det var värre i storstäderna mot landsbyggden och om sättet att fördela vacciner var rättvist/effektivt. Intressant sidopunkt att se om det fanns någon kommun som hade många fall men dålig vaccinationsmängd.

Jag hittade också olika intensivvårdsdata som skulle kunna jämföras med dödsfallen för att få en bild av hur effektiv intensivvården var, får göra mer dataanalys för att se om det fanns data för avlidna i vården eller om det enbart går att jämnföra dödsfall.

En till sak att titta på är åldersmätningarna som fanns och hur distributionen där ser ut, om valet att vaccinera äldre först rätt.

## KPI 1: Minskade dödsfall och svårt sjuka med vaccination?

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px


df_timeframe = pd.read_excel("Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name= "Vaccinationer tidsserie")
df_timeframe.head()

,Vecka,År,Region,Antal vaccinationer
0,52,2020,| Sverige |,2478
1,52,2020,Stockholm,289
2,52,2020,Uppsala,9
3,52,2020,Södermanland,116
4,52,2020,Östergötland,214


In [2]:
# Data on number of vaccinations divided into week/year/Region note that region does contain a region representing the whole country
df_timeframe.info()
df_timeframe.value_counts()
df_timeframe["Region"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2134 entries, 0 to 2133
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Vecka                2134 non-null   int64 
 1   År                   2134 non-null   int64 
 2   Region               2134 non-null   object
 3   Antal vaccinationer  2134 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 66.8+ KB


array(['| Sverige |', 'Stockholm', 'Uppsala', 'Södermanland',
       'Östergötland', 'Jönköping', 'Kronoberg', 'Kalmar', 'Gotland',
       'Blekinge', 'Skåne', 'Halland', 'Västra Götaland', 'Värmland',
       'Örebro', 'Västmanland', 'Dalarna', 'Gävleborg', 'Västernorrland',
       'Jämtland', 'Västerbotten', 'Norrbotten'], dtype=object)

### Notering
Eftersom jag vill jämnföra denna datan men dödsfall i riket och det är per vecka så läser jag in bladed veckodata riket. Jag har redan gjort en dataanalys av detta bladet i uppgift 1 och kommer därför inte göra det här.

In [3]:
# usefull columns år, veckonummer, Antal_nyaintensivvårdade_vecka, Antal_avlidna_vecka
df_cases_country = pd.read_excel("Folkhalsomyndigheten_Covid19.xlsx", sheet_name= "Veckodata Riket")
df_cases_country.head()

,år,veckonummer,Antal_fall_vecka,Antal_fall_100000inv_vecka,Antal_fall_100000inv_14dagar,Kum_antal_fall,Kum_fall_100000inv,Antal_nyaintensivvårdade_vecka,Kum_antal_intensivvårdade,Antal_avlidna_vecka,Antal_avlidna_milj_inv_vecka,Kum_antal_avlidna,Kum_antal_avlidna_milj_inv
0,2020,6,1,0,0,1,0,0,0,0,0.0,0,0.0
1,2020,7,0,0,0,1,0,0,0,0,0.0,0,0.0
2,2020,8,0,0,0,1,0,0,0,0,0.0,0,0.0
3,2020,9,13,0,0,14,0,0,0,0,0.0,0,0.0
4,2020,10,211,2,2,225,2,3,3,0,0.0,0,0.0


In [4]:
# cleaning up dataframe so that it only has data needed
df_timeframe = df_timeframe[df_timeframe["Region"] == "| Sverige |"]
df_timeframe["Vecka"] = df_timeframe["År"].astype(str) + "v" + df_timeframe["Vecka"].astype(str)
# could drop "Region" aswell here
df_timeframe.drop("År", axis = 1, inplace=True)
df_timeframe.reset_index(drop=True, inplace= True)
df_timeframe

,Vecka,Region,Antal vaccinationer
0,2020v52,| Sverige |,2478
1,2020v53,| Sverige |,16581
2,2021v1,| Sverige |,53258
3,2021v2,| Sverige |,132915
4,2021v3,| Sverige |,249713
...,...,...,...
92,2022v39,| Sverige |,24260220
93,2022v40,| Sverige |,24469813
94,2022v41,| Sverige |,24641177
95,2022v42,| Sverige |,24778109


In [10]:
# create "vecka" column in dataframe that matches the other dataframes column called "vecka", also drops any data that wont be needed for KPI
df_cases_country["Vecka"] = df_cases_country["år"].astype(str) + "v" + df_cases_country["veckonummer"].astype(str)
df_cases_country.drop(df_cases_country.columns.difference(["Vecka", "Antal_nyaintensivvårdade_vecka", "Antal_avlidna_vecka"]), axis = 1, inplace=True)
df_cases_country

,Antal_nyaintensivvårdade_vecka,Antal_avlidna_vecka,Vecka
0,0,0,2020v6
1,0,0,2020v7
2,0,0,2020v8
3,0,0,2020v9
4,3,0,2020v10
...,...,...,...
138,17,94,2022v39
139,20,98,2022v40
140,15,94,2022v41
141,20,70,2022v42


In [13]:
df = df_timeframe.join(df_cases_country.set_index("Vecka"), on = "Vecka")
df

,Vecka,Region,Antal vaccinationer,Antal_nyaintensivvårdade_vecka,Antal_avlidna_vecka
0,2020v52,| Sverige |,2478,206,643
1,2020v53,| Sverige |,16581,234,663
2,2021v1,| Sverige |,53258,185,644
3,2021v2,| Sverige |,132915,153,598
4,2021v3,| Sverige |,249713,125,513
...,...,...,...,...,...
92,2022v39,| Sverige |,24260220,17,94
93,2022v40,| Sverige |,24469813,20,98
94,2022v41,| Sverige |,24641177,15,94
95,2022v42,| Sverige |,24778109,20,70


In [17]:
fig = px.line(df, x= "Vecka", y=["Antal vaccinationer", "Antal_nyaintensivvårdade_vecka", "Antal_avlidna_vecka"], log_y=True)
fig.show()

#### Noteringar från denna data

Från starten av vaccinationen år 2020 vecka 52 till utplaningen av vaccinationskurvan när de flesta hade vaccinerats år 2021 vecka 27 så finns det en tydlig minskning i antalet dödsfall och intensivvårds fall vilket tyder på att vaccinet hade en effekt mot covid.
Det går även att dra slutsatser kring hur långvarig vaccineffekten är då det kommer en ny våg av dödsfall efter full vaccination. 

## KPI 2: Var det värst i de största städerna?

- vilka kommuner har varit bäst/sämst

In [19]:

df_vaccine_city = pd.read_excel("Folkhalsomyndigheten_Covid19_Vaccine.xlsx", sheet_name= "Vaccinerade kommun")
df_vaccine_city.head()

,KnKod,KnNamn,Antal_dos1,Antal_dos2,Andel_dos1,Andel_dos2
0,114,Upplands Väsby,33512,32223,0.808629,0.777526
1,115,Vallentuna,25412,24731,0.864001,0.840847
2,117,Österåker,36303,35248,0.871558,0.846230
3,120,Värmdö,34603,33533,0.870713,0.843789
4,123,Järfälla,57929,55504,0.806462,0.772703


In [22]:
df_vaccine_city.info()
df_vaccine_city["KnNamn"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   KnKod       290 non-null    int64  
 1   KnNamn      290 non-null    object 
 2   Antal_dos1  290 non-null    int64  
 3   Antal_dos2  290 non-null    int64  
 4   Andel_dos1  290 non-null    float64
 5   Andel_dos2  290 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 13.7+ KB


array(['Upplands Väsby', 'Vallentuna', 'Österåker', 'Värmdö', 'Järfälla',
       'Ekerö', 'Huddinge', 'Botkyrka', 'Salem', 'Haninge', 'Tyresö',
       'Upplands-Bro', 'Nykvarn', 'Täby', 'Danderyd', 'Sollentuna',
       'Stockholm', 'Södertälje', 'Nacka', 'Sundbyberg', 'Solna',
       'Lidingö', 'Vaxholm', 'Norrtälje', 'Sigtuna', 'Nynäshamn', 'Håbo',
       'Älvkarleby', 'Knivsta', 'Heby', 'Tierp', 'Uppsala', 'Enköping',
       'Östhammar', 'Vingåker', 'Gnesta', 'Nyköping', 'Oxelösund', 'Flen',
       'Katrineholm', 'Eskilstuna', 'Strängnäs', 'Trosa', 'Ödeshög',
       'Ydre', 'Kinda', 'Boxholm', 'Åtvidaberg', 'Finspång',
       'Valdemarsvik', 'Linköping', 'Norrköping', 'Söderköping', 'Motala',
       'Vadstena', 'Mjölby', 'Aneby', 'Gnosjö', 'Mullsjö', 'Habo',
       'Gislaved', 'Vaggeryd', 'Jönköping', 'Nässjö', 'Värnamo', 'Sävsjö',
       'Vetlanda', 'Eksjö', 'Tranås', 'Uppvidinge', 'Lessebo', 'Tingsryd',
       'Alvesta', 'Älmhult', 'Markaryd', 'Växjö', 'Ljungby', 'Högsby',
       '

In [55]:
df_data_city = pd.read_excel("Folkhalsomyndigheten_Covid19.xlsx", sheet_name= "Veckodata Kommun_stadsdel")
df_data_city.head()

,år,veckonummer,KnKod,KnNamn,Stadsdel,Kommun_stadsdel,tot_antal_fall_per10000inv,antal_fall_per10000_inv,tot_antal_fall,nya_fall_vecka
0,2020,1,1440,Ale,NaN,Ale,0.0,0.0,0,0
1,2020,2,1440,Ale,NaN,Ale,0.0,0.0,0,0
2,2020,3,1440,Ale,NaN,Ale,0.0,0.0,0,0
3,2020,4,1440,Ale,NaN,Ale,0.0,0.0,0,0
4,2020,5,1440,Ale,NaN,Ale,0.0,0.0,0,0


In [31]:
df_data_city["Stadsdel"].unique()

array([nan, 'Angered', 'Askim-Frölunda-Högsbo', 'Centrum', 'Lundby',
       'Majorna-Linné', 'Norra Hisingen', 'Västra Göteborg',
       'Västra Hisingen', 'Örgryte-Härlanda', 'Östra Göteborg', 'Fosie',
       'Husie', 'Hyllie', 'Kirseberg', 'Limhamn-Bunkeflo', 'Oxie',
       'Rosengård', 'Södra Innerstaden', 'Västra Innerstaden', 'Okänd',
       'Bromma', 'Enskede Årsta Vantör', 'Farsta', 'Hägersten Älvsjö',
       'Hässelby Vällingby', 'Kungsholmen', 'Norrmalm', 'Rinkeby Kista',
       'Skarpnäck', 'Skärholmen', 'Spånga Tensta', 'Södermalm',
       'Östermalm'], dtype=object)

två saker jag noterar här är att det finns kommunkoder för de olika kommunerna vilket är något som skulle kunna användas för sortering. Det andra är att i den ena datan är större kommuner indelade i stadsdelar. Beroende på hur datan ska användas så kan det slås ihop för att få data för hela städer eller ha separat för att få mer exakta mätpunkter.

In [60]:
# dataset 1 contains total vaccine data so need to combine dataset 2 so that it shows total amount instead of weekly count.
df_data_city.groupby("KnNamn").sum().reset_index()

C:\Users\Rasmus\AppData\Local\Temp\ipykernel_10372\1360930345.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,KnNamn,år,veckonummer,tot_antal_fall_per10000inv,antal_fall_per10000_inv
0,Ale,299098,3755,187053.0,2905.0
1,Alingsås,299098,3755,176845.0,2941.0
2,Alvesta,299098,3755,147060.0,2315.0
3,Aneby,299098,3755,131147.0,2026.0
4,Arboga,299098,3755,151230.0,2549.0
...,...,...,...,...,...
286,Österåker,299098,3755,151763.0,2492.0
287,Östhammar,299098,3755,116538.0,1851.0
288,Östra Göinge,299098,3755,121361.0,1797.0
289,Överkalix,299098,3755,120371.0,1803.0
